In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train_x = pd.read_csv("/kaggle/input/lish-moa/train_features.csv")
train_y = pd.read_csv("/kaggle/input/lish-moa/train_targets_scored.csv")
test_x = pd.read_csv("/kaggle/input/lish-moa/test_features.csv")
headers_y = train_y.columns.to_list()

In [ ]:
print(train_x.columns.to_series().groupby(train_x.dtypes).groups)
print("\n")
print(train_y.columns.to_series().groupby(train_y.dtypes).groups)
print("\n")
print(test_x.columns.to_series().groupby(test_x.dtypes).groups)
print("\n\n")
print(train_x["cp_type"].unique())
print("\n")
print(train_x["cp_dose"].unique())

In [ ]:
cp_type_ohe_train = pd.get_dummies(train_x["cp_type"], prefix="cp_type", dtype=float)
cp_dose_ohe_train = pd.get_dummies(train_x["cp_dose"], prefix="cp_dose", dtype=float)
cp_type_ohe_test = pd.get_dummies(test_x["cp_type"], prefix="cp_type", dtype=float)
cp_dose_ohe_test = pd.get_dummies(test_x["cp_dose"], prefix="cp_dose", dtype=float)
train_x = pd.concat([train_x, cp_type_ohe_train, cp_dose_ohe_train], axis=1, sort=False)
del train_x["cp_type"]
del train_x["cp_dose"]
test_x = pd.concat([test_x, cp_type_ohe_test, cp_dose_ohe_test], axis=1, sort=False)
del test_x["cp_type"]
del test_x["cp_dose"]

In [ ]:
# corr = train_x.corr()
# sns.heatmap(corr, 
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values)

In [ ]:
train_x_np = train_x.copy()
train_y_np = train_y.copy()
test_x_np = test_x.copy()
del train_x_np["sig_id"]
del train_y_np["sig_id"]
del test_x_np["sig_id"]
train_x_np = train_x_np.to_numpy()
train_x_np = np.asarray(train_x_np).astype('float32')
train_y_np = train_y_np.to_numpy()
train_y_np = np.asarray(train_y_np).astype('int32')
test_x_np = test_x_np.to_numpy()
test_x_np = np.asarray(test_x_np).astype('float32')
train_x_np = MinMaxScaler(feature_range=(0, 1)).fit_transform(train_x_np)
train_y_np = MinMaxScaler(feature_range=(0, 1)).fit_transform(train_y_np)
test_x_np = MinMaxScaler(feature_range=(0, 1)).fit_transform(test_x_np)

f = np.argmax(train_y_np, axis=1)
c = np.where(f == 0)[0]
d = np.random.choice(c, size=9034, replace=False)
s = train_y_np.copy()
train_y_np = np.delete(s, d, axis=0)
r = train_x_np.copy()
train_x_np = np.delete(r, d, axis=0)

In [ ]:
# print("Looking at distribution")
# print(np.histogram(np.argmax(train_y_np, axis=1), bins=206))

In [ ]:
model = tf.keras.models.Sequential()
model.add(
    tf.keras.layers.Input((train_x_np.shape[1],))
)
model.add(
    tf.keras.layers.Dense(units=train_x_np.shape[1], activation="relu")
)
model.add(
    tf.keras.layers.Dense(units=512, activation="relu")
)
model.add(
    tf.keras.layers.Dropout(0.5)
)
model.add(
    tf.keras.layers.Dense(units=1024, activation="relu")
)
model.add(
    tf.keras.layers.Dropout(0.5)
)
model.add(
    tf.keras.layers.Dense(units=512, activation="relu")
)
model.add(
    tf.keras.layers.Dropout(0.5)
)
model.add(
    tf.keras.layers.Dense(units=train_y_np.shape[1], activation="softmax")
)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
X_train, X_test, Y_train ,Y_test = train_test_split(train_x_np, train_y_np, test_size=0.2, random_state=56)

In [ ]:
model.fit(X_train, Y_train, epochs=20)
model.evaluate(X_test, Y_test)

In [ ]:
Y_hat = model.predict(test_x_np)

In [ ]:
Y_hat

In [ ]:
test_x["sig_id"]

In [ ]:
test_x_np.shape

In [ ]:
t = np.amax(Y_hat, axis=1)

In [ ]:
t

In [ ]:
result = np.full((3982,206), 0.0)

In [ ]:
for x in range(0, len(result)):
    u = np.where((Y_hat[x] == t[x]) == True)[0][0]
    result[x][u] = 1.0

In [ ]:
df = pd.DataFrame(data=result, columns=headers_y[1:])

In [ ]:
df["sig_id"] = test_x["sig_id"]

In [ ]:
df

In [ ]:
df.to_csv("submission.csv", index=False)